In [13]:
import pandas as pd
import math
from scipy import stats
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt
from pointpats import PointPattern, as_window
from pointpats import PoissonPointProcess as csr
from pointpats.geometry import (prepare_hull as _prepare_hull, area as _area,
    build_best_tree as _build_best_tree,
    prepare_hull as _prepare_hull,
    TREE_TYPES,
)
from scipy.spatial import distance
from pointpats.ripley import _prepare # very important to prepare data :)

![title](https://slideplayer.com/slide/14868061/90/images/52/adjusting+for+inhomogeneity.jpg)

![text](https://www.routledgehandbooks.com/assets/9781420082555/graphics/equ3_32.jpg)

In [ ]:
# since my dataset is badly formatted, need to flatten it before processing. Assuming it is called df:
df.columns = [['lat', 'long']]
def flatten(l):
    return [item for sublist in l for item in sublist]
df[['lat', 'long']] = df[['lat', 'long']].apply(flatten)
#then go from coords_to_array :)

In [14]:
def coords_to_array(df): 
    df['array'] = df.apply(lambda x: np.array(list(zip(x['lat'], x['long']))), axis = 1) 
    return df
simulated_companies = pd.read_pickle('/Users/gocchini/Desktop/paper_3/kinhom_work/simulated_companies.pkl')
simulated_companies.columns = [['lat', 'long']]
array_df = coords_to_array(simulated_companies)

In [3]:
array_df.head()

,lat,long,array
0,"[51.54218582497672, 51.589472314705674, 51.381...","[-0.0874064825256965, -0.24605053217241302, -0...","[[51.54218582497672, -0.0874064825256965], [51..."
1,"[51.55988941008741, 51.46151770699342, 51.4621...","[-0.2039481156403623, -0.10841744337054215, -0...","[[51.55988941008741, -0.2039481156403623], [51..."
2,"[51.56214130611294, 51.51280788687047, 51.3606...","[-0.12966170920812858, -0.2826523378177931, -0...","[[51.56214130611294, -0.12966170920812858], [5..."
3,"[51.56170641195442, 51.52023769828351, 51.4886...","[-0.166205136211394, -0.13644063967513753, -0....","[[51.56170641195442, -0.166205136211394], [51...."
4,"[51.4906797623787, 51.497014131477584, 51.5169...","[-0.1896979435260494, -0.17099360746609682, -0...","[[51.4906797623787, -0.1896979435260494], [51...."


# Sample controls and cases from simulations

In [17]:
list_of_arrays = simulated_companies['array'].values.tolist()
sampled_cases = [x[0][np.random.choice(x[0].shape[0], size=5, replace = False), :] for x in list_of_arrays]
simulated_companies['sample_cases'] = sampled_cases
sampled_controls = [x[0][np.random.choice(x[0].shape[0], size=5, replace = False), :] for x in list_of_arrays]
simulated_companies['sample_controls'] = sampled_controls

In [18]:
simulated_companies

,lat,long,array,sample_cases,sample_controls
0,"[51.54218582497672, 51.589472314705674, 51.381...","[-0.0874064825256965, -0.24605053217241302, -0...","[[51.54218582497672, -0.0874064825256965], [51...","[[51.55791592551092, -0.12359705773915552], [5...","[[51.50513764585078, -0.0868184504207491], [51..."
1,"[51.55988941008741, 51.46151770699342, 51.4621...","[-0.2039481156403623, -0.10841744337054215, -0...","[[51.55988941008741, -0.2039481156403623], [51...","[[51.53513137050367, -0.12319028440436142], [5...","[[51.58793688829641, -0.07808287146806137], [5..."
2,"[51.56214130611294, 51.51280788687047, 51.3606...","[-0.12966170920812858, -0.2826523378177931, -0...","[[51.56214130611294, -0.12966170920812858], [5...","[[51.362824386697795, -0.11427591636017442], [...","[[51.580283091327885, -0.21613723750152236], [..."
3,"[51.56170641195442, 51.52023769828351, 51.4886...","[-0.166205136211394, -0.13644063967513753, -0....","[[51.56170641195442, -0.166205136211394], [51....","[[51.48007573096543, -0.13911664160650206], [5...","[[51.49201166820216, -0.003933213300301719], [..."
4,"[51.4906797623787, 51.497014131477584, 51.5169...","[-0.1896979435260494, -0.17099360746609682, -0...","[[51.4906797623787, -0.1896979435260494], [51....","[[51.51083773315678, -0.2191692377894935], [51...","[[51.51919731361383, -0.07666137246809057], [5..."
...,...,...,...,...,...
995,"[51.63935421745579, 51.706750976828324, 51.542...","[-0.07261284013816416, -0.009181363548142474, ...","[[51.63935421745579, -0.07261284013816416], [5...","[[51.567650697539726, 0.13673630566787975], [5...","[[51.527406296091925, -0.14001612050780599], [..."
996,"[51.61569290725654, 51.63541106925196, 51.5147...","[0.11716453816526606, 0.122925519204793, -0.16...","[[51.61569290725654, 0.11716453816526606], [51...","[[51.487075503102986, -0.20527460906852257], [...","[[51.450477757240726, -0.12538147665732013], [..."
997,"[51.52412280586476, 51.491516413109416, 51.459...","[-0.14427792343017554, -0.3858098677670566, 0....","[[51.52412280586476, -0.14427792343017554], [5...","[[51.501406288547344, -0.2586429405957208], [5...","[[51.44769954750967, -0.1670212769452345], [51..."
998,"[51.57654497729315, 51.54095405036929, 51.5460...","[0.05455564471325014, -0.27824276403446013, -0...","[[51.57654497729315, 0.05455564471325014], [51...","[[51.54608562285099, -0.25536102723367116], [5...","[[51.534908076260514, -0.026684130261323324], ..."


# Get PDF from controls 

In [12]:
list_of_controls = simulated_companies['sample_controls'].values.tolist()

x = []
for coordinata in list_of_controls:
    c = coordinata[0][0][0]
    x.append(c)
x = np.array(x)

y = []
for coordinata in list_of_controls:
    c = coordinata[0][0][1]
    y.append(c)
y = np.array(y)

values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)

# Define functions to find lambdas and calculate kinhom

In [23]:
def find_lambda(kernel, x, y):
    values = np.vstack([x, y])
    return kernel.pdf(values)

def fun_find_kinhom(distance, support, multiplied_lambdas):
    kinhom_values = []
    for x in support:
        if x > distance:
            x = 1/multiplied_lambdas
        else: 
            x = 0 
        kinhom_values.append(x)
    return np.array(kinhom_values)

# Get distances and calculate kinhom on cases

In [19]:
cases_array = simulated_companies['sample_cases'].values.tolist()

In [21]:
example = cases_array[0][0]

In [22]:
example

array([[51.55791593, -0.12359706],
       [51.50549456, -0.14876744],
       [51.48257208, -0.11538145],
       [51.43544935, -0.24095732],
       [51.51235937, -0.29067871]])

# Define distances at which to asses Kinhom

In [26]:
support = [0.        , 0.00584394, 0.01168788, 0.01753182, 0.02337576,
       0.0292197 , 0.03506364, 0.04090759, 0.04675153, 0.05259547]

In [38]:
def find_lambdas_and_distances(array_of_simulated_coordinates, support):
    all_pairs = [(a, b) for idx, a in enumerate(array_of_simulated_coordinates) for b in array_of_simulated_coordinates[idx + 1:]]
    df = pd.DataFrame(all_pairs)
    df.columns = [['x', 'y']]
    df['lambda_values_x'] = df.apply(lambda x: find_lambda(kernel, x['x'][0], x['x'][1]), axis = 1) 
    df['lambda_values_y'] = df.apply(lambda x: find_lambda(kernel, x['y'][0], x['y'][1]), axis = 1)
    df['inhom_lambda'] = df.apply(lambda x: x['lambda_values_x']*x['lambda_values_y'], axis =1) 
    df['distance'] = df.apply(lambda x: distance.euclidean(x['x'], x['y']), axis = 1)
    support_names = map(str, support)
    support_pd = pd.DataFrame(zip(support_names, support),
               columns =['distance_name', 'distance'])
    support_pd = support_pd.T
    support_pd.columns = support_pd.iloc[0]
    support_pd = support_pd.iloc[1: , :]
    support_pd = pd.DataFrame(np.repeat(support_pd.values, len(df), axis = 0))
    support_pd.columns = support_pd.iloc[0]
    support_pd['support'] = support_pd.values.tolist()
    df = pd.concat([df, support_pd['support']], axis=1)
    df.columns = [['x', 'y', 'lambda_values_x','lambda_values_y', 'multiplied_lambdas', 'distance', 'support']]
    df['kinhom'] = df.apply(lambda x: fun_find_kinhom(x['distance'], x['support'], x['multiplied_lambdas']), axis = 1)
    n_pairs_less_than_d = df['kinhom'].values.tolist()
    internal_argument = np.sum(n_pairs_less_than_d, 0)
    return df, n_pairs_less_than_d, internal_argument

In [39]:
kinhoms = find_lambdas_and_distances(example, support)

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_20476/3229891459.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(kinhom_values)


In [40]:
kinhoms[0]

,x,y,lambda_values_x,lambda_values_y,multiplied_lambdas,distance,support,kinhom
0,"[51.55791592551092, -0.12359705773915552]","[51.505494555612685, -0.14876743994497088]",[14.588857190681457],[25.468196964731494],[371.5518884226147],0.058151,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[51.55791592551092, -0.12359705773915552]","[51.48257208273024, -0.11538144979607867]",[14.588857190681457],[19.389837770694307],[282.87557418714056],0.075790,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[51.55791592551092, -0.12359705773915552]","[51.43544934504668, -0.24095732470290188]",[14.588857190681457],[5.888415633576478],[85.90525475762331],0.169622,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[51.55791592551092, -0.12359705773915552]","[51.512359368311515, -0.29067870968137177]",[14.588857190681457],[7.567767205354708],[110.40507501124235],0.173181,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[51.505494555612685, -0.14876743994497088]","[51.48257208273024, -0.11538144979607867]",[25.468196964731494],[19.389837770694307],[493.8242074582328],0.040498,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, [0.0020250121093639967],..."
5,"[51.505494555612685, -0.14876743994497088]","[51.43544934504668, -0.24095732470290188]",[25.468196964731494],[5.888415633576478],[149.96732916612993],0.115781,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,"[51.505494555612685, -0.14876743994497088]","[51.512359368311515, -0.29067870968137177]",[25.468196964731494],[7.567767205354708],[192.7373857692093],0.142077,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"[51.48257208273024, -0.11538144979607867]","[51.43544934504668, -0.24095732470290188]",[19.389837770694307],[5.888415633576478],[114.17542386146803],0.134126,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"[51.48257208273024, -0.11538144979607867]","[51.512359368311515, -0.29067870968137177]",[19.389837770694307],[7.567767205354708],[146.7377783982084],0.177810,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,"[51.43544934504668, -0.24095732470290188]","[51.512359368311515, -0.29067870968137177]",[5.888415633576478],[7.567767205354708],[44.56215872327803],0.091583,"[0.0, 0.00584394, 0.01168788, 0.01753182, 0.02...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [42]:
kinhoms[1]

[[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, array([0.00202501]), array([0.00202501]),
         array([0.00202501])], dtype=object)],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]]

In [41]:
kinhoms[2]

array([[0, 0, 0, 0, 0, 0, 0, array([0.00202501]), array([0.00202501]),
        array([0.00202501])]], dtype=object)

# Calculate hull on full set

In [50]:
coords_array = list_of_arrays[0][0]
hull = _prepare_hull(coords_array)

In [51]:
one_divided_area = 1/_area(hull)
K = kinhoms[2]*one_divided_area

In [52]:
K

array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, array([0.01085781]),
        array([0.01085781]), array([0.01085781])]], dtype=object)

# Transform to L 

In [28]:
def L_function(list_of_estimates):
    L = [math.sqrt(x/math.pi) for x in list_of_estimates]
    return L
#kinhom_estimates['l_estimates'] = kinhom_estimates['kinhom_estimate'].apply(L_function)